In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import json

import IPython
import requests
import dotenv
import pandas as pd
import numpy as np

from backend.location import Location, make_grid, spherical_distance
from backend import gmaps

dotenv.load_dotenv()

GMAPS_API_KEY = os.environ["GMAPS_API_KEY"]

In [ ]:
# zurich = Location(47.364, 8.5415)  # close to Bürkliplatz
# zurich = Location(47.355, 8.545)  # close to Chinagarten
zurich = Location(47.3704468136481,8.54363623046875)
zoom = 14

image = gmaps.get_static_map(zurich, zoom, markers=[zurich])

IPython.display.Image(image)

In [ ]:
str(zurich)

In [ ]:
response = requests.get(f"https://maps.googleapis.com/maps/api/geocode/json?latlng={zurich}&key={GMAPS_API_KEY}")

data = response.json()["results"]
locations = [
    Location(x["geometry"]["location"]["lat"], x["geometry"]["location"]["lng"])
    for x in data
]

In [ ]:
zoom = 14

for i, location in enumerate(locations):
    print(i, data[i]["types"])
    image = gmaps.get_static_map(zurich, zoom, markers=[zurich, location])
    display(IPython.display.Image(image))

In [ ]:
buerkliplatz = Location(47.369, 8.5415)
grid = make_grid(buerkliplatz, zoom=15, size=9)

image = gmaps.get_static_map(buerkliplatz, zoom=15, markers=grid)
IPython.display.Image(image)

In [ ]:
def resolve(location: Location):
    response = requests.get(
        f"https://maps.googleapis.com/maps/api/geocode/json?"
        f"latlng={location}&key={GMAPS_API_KEY}"
    )
    data = response.json()

    if data["status"] != "OK":
        raise ValueError(f"Got non-OK status when resolving {location}. Got: {data}")

    # I also tried "street_address", which sounds like what we'd actually want,
    # but this left multiple markers in the lake.
    result_types = ["route", "point_of_interest"]
    filtered_results = [
        x for x in data["results"] if any(t in x["types"] for t in result_types)
    ]

    if not filtered_results:
        raise ValueError(f"No location found when resolving {location}. Got: {data}")

    resolution = filtered_results[0]

    return {
        "location": Location(
            resolution["geometry"]["location"]["lat"],
            resolution["geometry"]["location"]["lng"],
        ),
        "place_id": resolution["place_id"],
        "types": resolution["types"],
    }

In [ ]:
import tqdm.auto

resolved_grid = [resolve(location)["location"] for location in tqdm.auto.tqdm(grid)]

In [ ]:
image = gmaps.get_static_map(buerkliplatz, zoom=15, markers=resolved_grid)
IPython.display.Image(image)